In [59]:
# Import required models
from tensorflow.keras import layers, Sequential, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [60]:
#Initialize image data generator with generator wuth rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)


In [61]:
#preprocess all test images
train_generator = train_data_gen.flow_from_directory(
    'data/train',
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.


In [62]:
validation_generator = train_data_gen.flow_from_directory(
    'data/test',
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
)

Found 7178 images belonging to 7 classes.


In [63]:
#create moedel architecture

emotion_model = models.Sequential ([
    layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)),
    layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25), 

    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(7, activation='softmax')

])

c:\Users\souri\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [64]:
emotion_model.compile(
    optimizer='adam' ,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [65]:
emotion_model_info = emotion_model.fit(
    train_generator,
    steps_per_epoch=28709 // 64,  # Number of training batches
    epochs=100,                     # Number of epochs to train
    validation_data=validation_generator,
    validation_steps=7178 // 64    # Number of validation batches
)


Epoch 1/100


c:\Users\souri\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


448/448 ━━━━━━━━━━━━━━━━━━━━ 52s 111ms/step - accuracy: 0.2758 - loss: 1.7675 - val_accuracy: 0.4411 - val_loss: 1.4523
Epoch 2/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 80us/step - accuracy: 0.4688 - loss: 1.5547 - val_accuracy: 0.5000 - val_loss: 1.5622
Epoch 3/100


c:\Users\souri\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


448/448 ━━━━━━━━━━━━━━━━━━━━ 52s 115ms/step - accuracy: 0.4445 - loss: 1.4269 - val_accuracy: 0.5142 - val_loss: 1.2761
Epoch 4/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 44us/step - accuracy: 0.5000 - loss: 1.3775 - val_accuracy: 0.4000 - val_loss: 1.3430
Epoch 5/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 53s 118ms/step - accuracy: 0.5199 - loss: 1.2570 - val_accuracy: 0.5451 - val_loss: 1.1767
Epoch 6/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 74us/step - accuracy: 0.5000 - loss: 1.0228 - val_accuracy: 0.9000 - val_loss: 0.8314
Epoch 7/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 56s 124ms/step - accuracy: 0.5593 - loss: 1.1716 - val_accuracy: 0.5660 - val_loss: 1.1410
Epoch 8/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 56us/step - accuracy: 0.5938 - loss: 1.0849 - val_accuracy: 0.5000 - val_loss: 1.3224
Epoch 9/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 56s 124ms/step - accuracy: 0.5835 - loss: 1.1013 - val_accuracy: 0.5738 - val_loss: 1.1212
Epoch 10/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 48us/step - accuracy: 0.5781 - loss: 1.3224 - val

In [66]:
#save model sttructure is json file
model_json = emotion_model.to_json()
with open("emotion_model.json","w") as json_file:
    json_file.write(model_json)

#save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.weights.h5')